# LLM Basics

For this workshop, we will build a chatbot to answer questions about Git. 

In this notebook, we will see how you can easily invoke an LLM to answer questions, and we will explore some of the shortcomings that make retrieval-augmented generation appealing.

## Question answering with LLMs

Let's start by asking some simple questions to an LLM. This will also introduce `langchain`, which provides lightweight but useful wrappers to work with the many libraries and providers that get chained together to build an LLM application. See https://www.langchain.com/ or try https://www.llamaindex.ai/ for an alternative.

We will use https://huggingface.co/HuggingFaceH4/zephyr-7b-beta as our LLM, but you can easily switch out to any other model. To try out other models, replace the `repo_id` with another model from https://huggingface.co/models, or see https://python.langchain.com/docs/integrations/llms/ to use a non-Hugging Face Provider (for example, OpenAI).

In [1]:
from langchain import hub
from langchain_community.llms import HuggingFaceEndpoint

llm = HuggingFaceEndpoint(repo_id="HuggingFaceH4/zephyr-7b-beta")

/Users/dave/micromamba/envs/mlops-for-llms-workshop/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /Users/dave/.cache/huggingface/token
Login successful


Let's ask some questions about Git.

In [2]:
llm.invoke("What is a branch?")

'\nA branch is a structure on a tree that grows from the trunk or a larger branch. Branches provide support for leaves, flowers, and fruits, and help the tree to spread out and expand its reach. The shape and arrangement of branches can vary widely depending on the species of tree. Some trees have broad, spreading branches, while others have narrow, upright branches. The angle and distance between branches can also affect the overall shape and growth pattern of the tree. Branches can play a crucial role in the ecology and habitat of a forest, providing shelter, food, and nesting sites for a variety of plant and animal species.'

Of course, the LLM doesn't know we are asking about Git. This highlights one of the upsides of using RAG -- it will provide context for the LLM to use.

Let's try again by providing more context.

In [3]:
llm.invoke("What is a Git branch?")

'\n\nA Git branch is a lightweight movable pointer (a special type of reference) to a commit in a Git repository.\n\nA Git branch is a named pointer to a commit. A branch doesn’t contain the commit objects themselves; it just contains a pointer to the commit object.\n\nGit branches are used to manage and organize work in a Git repository. Branches can be created, deleted, and moved around as needed.\n\nThere are two types of branches in Git: local branches and remote branches. Local branches are created and managed within a local Git repository, while remote branches are created and managed on a remote Git server or repository.\n\nWhen working on a Git repository, it’s common to create a branch for each new feature, bug fix, or experimental change. This allows multiple changes to be worked on independently and then merged back into the main branch (usually called the “main” or “master” branch) when they’re ready.\n\nOverall, Git branches are a powerful and flexible tool for managing an

Now that the question provides the context that we are asking about Git, the LLM gives a reasonable response.

Since Git is well documented and the LLM has likely been trained on a lot of existing content about Git, the above response might be good enough and we don't need to provide more info. When asked about less publicly available or more recent information, LLMs may struggle without additional context.

Let's try a question about the conference.

In [4]:
llm.invoke("When is the Uphill Conference?")

'\nThe Uphill Conference is held every year in June.\nThe 2022 Uphill Conference will be held on June 14-16 at the Royal Sonesta Harbor Court Baltimore Hotel.\nHow do I register for the Uphill Conference?\nTo register for the conference, you must be a current Uphill Member.\nOnce you are a member, you will receive an email with instructions on how to register for the conference.\nCan I attend the Uphill Conference as a non-member?\nThe Uphill Conference is exclusively for Uphill Members.\nIf you are interested in becoming a member, please visit our membership page.\nWhat is included in the conference registration fee?\nThe conference registration fee includes access to all conference sessions, conference materials, meals, and networking events.\nWhat topics will be covered at the Uphill Conference?\nThe Uphill Conference covers a wide range of topics related to the unique challenges and opportunities faced by manufacturers and distributors in the uphill market.\nSome of the topics that

Not only does this provide wrong dates from years ago, it doesn't even reference the correct conference, and it provides a lot of irrelevant info. Let's add context to help the LLM provide more relevant and factual information.

## How RAG provides context to LLMs

RAG retrieves relevant information to the question and injects it into the prompt provided to the LLM. Let's add the relevant info about the conference and see if it helps.

In [5]:
prompt = hub.pull("rlm/rag-prompt").messages[0].prompt

In [6]:
prompt

PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:")

In [7]:
question = "When is the Uphill Conference?"

context = """
4th Uphill Conf - May 16 & 17, 2024 – Bern, Switzerland

Uphill Conf is back with a new topic: We will explore the profound and lasting impact of Artificial Intelligence and Machine Learning on software engineering."
"""

input = prompt.invoke({"question": question, "context": context})
llm.invoke(input.text)

' The 4th Uphill Conference is scheduled for May 16 and 17, 2024, in Bern, Switzerland, where the focus will be on the impact of Artificial Intelligence and Machine Learning on software engineering.'

Now the response is helpful and accurate! It may seem obvious since we provided the exact information we wanted in the answer, but this is all RAG does: inject relevant information into the prompt sent to the LLM. Including the most relevant information is the key.

Next, we will see how to parse and retrieve relevant information for the RAG application. We will keep using Git questions since it will be easy to understand and evaluate the responses, but keep in mind that the advantages of using RAG will be more pronounced in other contexts where the LLM has less information about the subject.